In [4]:
import requests
from pprint import pprint
import pandas as pd
from pandas import ExcelWriter
import io, time, json

##Below block of code is used to extract the data from the Zomato API.

headers = {'Content-Type': 'application/json', 'user-key': '592789033e4cf0e384b54c1ec424c706'}
offset = 0
restaurantNameList = []
ratingList = []
ratingTextList = []
votesList = []
totalData = []
urlList = []
addressList = []
sourceList = []
while offset < 100:
    url = 'https://developers.zomato.com/api/v2.1/search?entity_id=291&entity_type=city&start='+str(offset)+'&count=20'
    Response = requests.get(url , headers=headers)
    offset += 20
    Response_Json = Response.json()
    Array_length = Response_Json['restaurants']
    for i in range(len(Array_length)):
        totalData.append(Response_Json['restaurants'][i])
        restaurantNameList.append(Response_Json['restaurants'][i]['restaurant']['name'])
        ratingList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['aggregate_rating'])
        ratingTextList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['rating_text'])
        votesList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['votes'])
        urlList.append(Response_Json['restaurants'][i]['restaurant']['url'])
        addressList.append(Response_Json['restaurants'][i]['restaurant']['location']['address'])
        sourceList.append('Zomato')
df1 = pd.DataFrame({'AllData':totalData})
df2 = pd.DataFrame({'Restaurant Name':restaurantNameList,'Rating':ratingList, 'Rating Text':ratingTextList, 'Votes':votesList, 'Url':urlList, 'Address': addressList, 'Source': sourceList})
writer = pd.ExcelWriter('../DataFiles/Final_Data.xlsx' , engine='xlsxwriter')
df1.to_excel(writer,'ZomatoRaw',index=False)
df2.to_excel(writer,'ZomatoClean',index=False)
writer.save()


In [5]:
##The below block of code is used to read the API Key required for extracting data from Yelp fusion API.

def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

In [9]:
##The function below is used to retrieve all the restaurants from the YELP API.

def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    
    offsetno=0
    total =0
    output=[]
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    while (offsetno <= total):
        params = { #parameters are case sensitive!
            "location": query,
            "limit": 20,
            "offset": offsetno,
            "categories": "restaurants"
            
        }
    
        response = requests.get('https://api.yelp.com/v3/businesses/search',
                            headers=headers, params=params)
        result = response.json()
        try:
            output.append( result['businesses'])
        except KeyError:
            print(result)
            return output
        offsetno =offsetno + 20
        total = result['total']
        print (offsetno)
        #print (len(output))
    return(output)

api_key = read_api_key('api_key.txt')
data = all_restaurants(api_key, 'Miami')
new_data = {}
new_data["name"]=[]
new_data["rating"]=[]
new_data["price"]=[]
new_data["review_count"]=[]
new_data["url"]=[]
new_data["category"]=[]
new_data["zip"]=[]
alldata=[]
for i in data:
    for each in i:
        new_data["name"].append(each["name"])
        new_data["rating"].append(each["rating"])
        new_data["url"].append(each["url"])
        try:
            new_data["price"].append(each["price"])
        except KeyError:
            new_data["price"].append(None)

        new_data["review_count"].append(each["review_count"])
        new_data["category"].append(each["categories"][0]["title"])
        new_data["zip"].append(each["location"]["zip_code"])
        alldata.append(each)
df4 = pd.DataFrame(alldata)
df4.to_excel(writer,'YelpRaw',index=False)
df3 = pd.DataFrame(new_data)
df3.to_excel(writer,'YelpClean',index=False)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}


In [8]:
#df.to_csv("Miami_Yelp.csv",index=False)
#df_all = pd.DataFrame(alldata)
#df_all.to_csv("Miami_raw.csv",index=False)
#df = pd.read_excel("/Users/srossgupta/Desktop/Data Focussed Python/HungryGator/DataFiles/Project_prototype_datafiles.xlsx",sheet_name="MergedData")

20
1
40
2
60
3
80
4
100
5
120
6
140
7
160
8
180
9
200
10
220
11
240
12
260
13
280
14
300
15
320
16
340
17
360
18
380
19
400
20
420
21
440
22
460
23
480
24
500
25
520
26
540
27
560
28
580
29
600
30
620
31
640
32
660
33
680
34
700
35
720
36
740
37
760
38
780
39
800
40
820
41
840
42
860
43
880
44
900
45
920
46
940
47
960
48
980
49
1000
50
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}


In [ ]:
##The below code needs to be executed to clean the TripAdvisor CSV file and create a raw and clean dataset out of it.

data = pd.read_csv('../DataFiles/TripAdvisor_Data.csv')
cleaned_data = data[data["City"]=="Miami"]
cleaned_data = cleaned_data.loc[:,["Restaurant ID", "Restaurant URL","Name","Address","Ranking","Total Review"]]
#path = r"C:/Users/devdi/Desktop/CMU/Final Mini/Python for Data Science/Project/Data.xlsx"
#writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
data.to_excel(writer, sheet_name = ' TripAdvisor_raw')
cleaned_data.to_excel(writer, sheet_name = 'TripAdvisor_clean')
writer.save()
writer.close()

